In [6]:
import asn1
from pygost import gost34112012256
from random import randint
from Crypto.Util.number import inverse

a = -1
b = 53956679838042162451108292176931772631109916272820066466458395232513766926866
p = 57896044625259982827082014024491516445703215213774687456785671200359045162371
r = 28948022312629991413541007012245758222850495633896873081323396140811733708403
x = 12933162268009944794066590054824622037560826430730236852169234350278155715869
y = 18786030474197088418858017573086015439132081546303111294023901101650919011383
q = r


def encode_gost_sign(s, r, Q):
    xQ = lift(Q[0])
    yQ = lift(Q[1])
    
    encoder = asn1.Encoder()
    encoder.start()
    
    encoder.enter(asn1.Numbers.Sequence)
    encoder.enter(asn1.Numbers.Set)
    
    encoder.enter(asn1.Numbers.Sequence)  # first key
    encoder.write(b'\x80\x06\x07\x00', asn1.Numbers.OctetString)
    encoder.write(b'34.10-2018', asn1.Numbers.UTF8String)
 
    encoder.enter(asn1.Numbers.Sequence)  # public key
    encoder.write(xQ, asn1.Numbers.Integer)
    encoder.write(yQ, asn1.Numbers.Integer)
    encoder.leave()
    
    encoder.enter(asn1.Numbers.Sequence)  # cryptosystem params
    
    encoder.enter(asn1.Numbers.Sequence)  # field params
    encoder.write(p, asn1.Numbers.Integer)
    encoder.leave()
    
    encoder.enter(asn1.Numbers.Sequence)  # curve params
    encoder.write(a, asn1.Numbers.Integer)
    encoder.write(b, asn1.Numbers.Integer)
    encoder.leave()
    
    encoder.enter(asn1.Numbers.Sequence)  # group generator
    encoder.write(x, asn1.Numbers.Integer)
    encoder.write(y, asn1.Numbers.Integer)
    encoder.leave()
    
    encoder.write(q, asn1.Numbers.Integer)
    
    encoder.leave()
    
    encoder.enter(asn1.Numbers.Sequence)  # message sign
    encoder.write(r, asn1.Numbers.Integer)
    encoder.write(s, asn1.Numbers.Integer)
    encoder.leave()
    
    encoder.leave()
    
    encoder.enter(asn1.Numbers.Sequence)
    encoder.leave()
    
    encoder.leave()
    encoder.leave()
    
    return encoder.output()


def asn_parse(decoder, ints):
    while not decoder.eof():
        try:
            tag = decoder.peek()
            if tag.nr == asn1.Numbers.Null:
                break
            if tag.typ == asn1.Types.Primitive:
                tag, value = decoder.read()
                if tag.nr == asn1.Numbers.Integer:
                    ints.append(value)
            elif tag.typ == asn1.Types.Constructed:
                decoder.enter()
                asn_parse(decoder, ints)
                decoder.leave()
        except asn1.Error:
            print("Can't parse file")
            break

    return ints


def decode_gost_sign(data):
    decoder = asn1.Decoder()
    decoder.start(data)
    ints = []
    ints = asn_parse(decoder, ints)
    return ints[0], ints[1], ints[2], ints[3], ints[4], ints[5], ints[6], ints[7], ints[8], ints[9]


def verify_sign(filename):
    global q, r
    
    with open('sign.asn1', 'rb') as file:
        encoded_data = file.read()
        
    xQ, yQ, p, A, B, xP, yP, q, r, s = decode_gost_sign(encoded_data)

    with open(filename, 'rb') as file:
        data = file.read()
        
    if r <= 0 or r >= q or s <= 0 or s >= q:
        print('File was changed')
        return 
        
    hash = gost34112012256.new(data).digest()
    h = int.from_bytes(hash, 'big')
    
    e = h % q
    
    if e == 0:
        e = 1
    
    v = inverse(e, q)
    z2 = (-r  * v) % q
    z1 = (s * v) % q
    
    E = EllipticCurve(GF(p), [A,B])
    P = E(xP, yP)
    Q = E(xQ, yQ)
    
    C = z1 * P + z2 * Q
    xc = C[0]
    xc = lift(xc)
    R  = xc % q
    
    if R == r:
        print("File wasn't changed")
    else:
        print("File was changed")
    
    
def create_sign(filename):
    E = EllipticCurve(GF(p), [a, b])
    P = E(x, y)
    
    with open(filename, 'rb') as file:
        data = file.read()
    
    hash = gost34112012256.new(data).digest()
    h = int.from_bytes(hash, 'big')
    e = h % q
    
    if e == 0:
        e = 1
    
    d = randint(0, q)
    Q = d * P
    
    print(f'd = {d}')
    print('%x' % h)
    
    
    while True:
        k = randint(0, q)
        C = k * P
        xc = C[0]
        xc = lift(xc)
        
        r = xc % q
        if r == 0:
            continue
            
        s = (r * d + k * e) % q
        if s == 0:
            continue
        
        encoded_data = encode_gost_sign(s, r, Q)
        
        with open('sign.asn1', 'wb') as file:
            file.write(encoded_data)
        
        return


def main():
    act = int(input('1 - create signature\n2 - verify signature\n'))
    file = input('Enter filename: ')
    
    if act == 1:
        create_sign(file)
    elif act == 2:
        verify_sign(file)
    else:
        print('Incorrect action!')
    
main()

1 - create signature
2 - verify signature
2
Enter filename: msg.txt
File was changed
